# Chicago Weather Data Collection
**Purpose**: Fetch hourly weather data from Open-Meteo API for Chicago coordinates  
**Output**: Weather DataFrame with temperature, wind, precipitation for taxi trip correlation  
**Date Range**: Historical data (T-2 months) to match taxi data availability

In [ ]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Chicago coordinates (matches taxi pickup/dropoff areas)
CHICAGO_LAT = 41.85
CHICAGO_LON = -87.65

In [ ]:
# Get date 2 months ago for historical weather data
target_date = datetime.now() - relativedelta(months=2)
date_str = target_date.strftime("%Y-%m-%d")

print(f"Fetching weather data for Chicago on: {date_str}")

In [ ]:
# Open-Meteo API endpoint for historical weather
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": CHICAGO_LAT,
    "longitude": CHICAGO_LON,
    "start_date": date_str,
    "end_date": date_str,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

# Fetch weather data
response = requests.get(url, params=params)
response.raise_for_status()
weather_data = response.json()

print(f"API Response Status: {response.status_code}")

In [ ]:
# Transform API response to DataFrame for taxi correlation
weather_df = pd.DataFrame({
    "datetime": pd.to_datetime(weather_data["hourly"]["time"]),
    "temperature": weather_data["hourly"]["temperature_2m"],
    "wind_speed": weather_data["hourly"]["wind_speed_10m"], 
    "rain": weather_data["hourly"]["rain"],
    "precipitation": weather_data["hourly"]["precipitation"]
})

print(f"Weather data retrieved: {len(weather_df)} hourly records")
weather_df.head()

In [ ]:
# Save weather data for taxi correlation pipeline
import os
output_path = "csv/chicago_weather.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
weather_df.to_csv(output_path, index=False)
print(f"Weather data saved: {output_path}")